# Step by step notebook to get Claude hooked up to Fused MCP servers

This notebook walks you through setting up an MCP server locally with Claude's desktop app

Requirements:
- Have Claude desktop app installed
- Python 3.13
- `uv` installed

By the end of this notebook you'll have everything you need to run CLaude locally with _any_ Python code you pass as a Fused User Defined Function, all running locally

In [2]:
import fused
import json
import os
import time

In [3]:
commit = "12e5d3b"
common = fused.load(f"https://github.com/fusedio/udfs/tree/{commit}/public/common").utils

In [4]:
# All defaults paths needed right now
PATH_TO_CLAUDE_CONFIG = "/Users/maximelenormand/Library/Application Support/Claude/claude_desktop_config.json"
CLAUDE_APP_PATH = "/Applications/Claude.app"

In [4]:
# Since we're only doing engine='local' caching isn't even needed
@fused.udf(cache_max_age='0s')
def udf():
    """Simply return the current time in UTC"""
    import time
    import pandas as pd
    from datetime import datetime

    return pd.DataFrame({"current_utc_time": [
        datetime.fromtimestamp(time.time()).strftime("%Y-%m-%d %H:%M:%S")
    ]})

In [5]:
# Only engine=local for no account users
fused.run(udf, engine='local')

,current_time
0,2025-03-18 09:17:29


In [8]:
os.path.exists("./agents.json")

True

In [9]:
common.save_to_agent(
    agent_json_path = "./agents.json",
    udf = udf,
    udf_name = "get_current_time",
    mcp_metadata = {
        "description": "UDF that returns the current datetime",
        "parameters": "",
    }
)

In [17]:
# ?common.save_to_agent

Signature:
common.save_to_agent(
    agent_json_path: 'str',
    udf: 'AnyBaseUdf',
    udf_name: 'str',
    mcp_metadata: 'dict[str, Any]',
    overwrite: 'bool' = True,
)
Docstring:
Save UDF to agent of udf_ai directory
Args:
    agent_json_path (str): Absolute path to the agent.json file
    udf (AnyBaseUdf): UDF to save
    udf_name (str): Name of the UDF
    mcp_metadata (dict[str, Any]): MCP metadata
    overwrite (bool): If True, overwrites any existing UDF directory with current `udf`
File:      ~/Library/CloudStorage/Dropbox/Mac/Documents/repos/udf-test/utils.py
Type:      function

In [ ]:
#TODO: Creating a new agent with UDFs created inline here for ex:
# Get UTC time
# UTC to local time


In [12]:
# Checking the agents.json file
json.load(open("./agents.json"))

{'agents': [{'name': 'get_current_time', 'udfs': ['get_current_time']}]}

In [5]:
common.generate_local_mcp_config(
    config_path=PATH_TO_CLAUDE_CONFIG,
    agents_list = ["fused_docs"],
    repo_path= os.getcwd(), # Local current path, assuming repo is here
    uv_path = "/Users/maximelenormand/.local/bin/uv",
)

In [6]:
# Reading claude config to confirm
json.load(open(PATH_TO_CLAUDE_CONFIG))

{'mcpServers': {'fused_docs': {'command': '/Users/maximelenormand/.local/bin/uv',
   'args': ['run',
    '--directory',
    '/Users/maximelenormand/Library/CloudStorage/Dropbox/Mac/Documents/repos/udf-test',
    'run.py',
    '--runtime=local',
    '--udf-names=list_public_udfs,reading_fused_docs,get_current_time',
    '--name=fused_docs']}}}

In [5]:
# Now restart Claude!

In [8]:
app_name = CLAUDE_APP_PATH.split("/")[-1]  # Extracts "Claude.app"
print(f"{app_name=}")
try:
    os.system(f"pkill -f '{app_name}'")  # Kill the app
    print(f"Killed {app_name}")
    time.sleep(2)  # Wait for shutdown
except Exception as e:
    print(f"Claude wasn't running, so no need to kill it")

print(f"Restarting {app_name}")
os.system(f"open -a '{CLAUDE_APP_PATH}'")  # Restart Claude

app_name='Claude.app'
Killed Claude.app
Restarting Claude.app


0